# Capstone Project - Whats the best hotel to stay at in Shanghai?
## (Closest to the best rated restaurants)
### *Lawrence Sheed*

---

###Do our Setup Code

In [2]:
#Disable pip's bloody annoying warnings
%env _PIP_LOCATIONS_NO_WARN_ON_MISMATCH=1 

#disable python warnings
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

#install libraries needed
!pip install beautifulsoup4
!pip install requests
!pip install pandas
!pip install numpy
!pip install matplotlib
!pip install geopy
!pip install geocoder

#import libraries
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import json
import folium 
from bs4 import BeautifulSoup  

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe


# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
    

from sklearn.cluster import KMeans
from IPython.display import display_html
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
import geocoder # import geocoder

print('Libraries imported.')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)



env: _PIP_LOCATIONS_NO_WARN_ON_MISMATCH=1
Libraries imported.


### Login to FourSquare w/Credentials

In [261]:
#Foursquare login BS

CLIENT_ID = 'MGHGZ0OLVSC4UT1XM4Z5IEH0UKRQ5YOT5XZ42SLUWXDDPUEK' # your Foursquare ID
CLIENT_SECRET = 'COFCZNWCRD1HKCRIJ5B3M1XP1TLQ04HEVPJASFWO1PGILHSH' # your Foursquare Secret
ACCESS_TOKEN = '0JXEDHYX0LFM1M3P1XLVEV0TNGWOGPVCE4PKVR3E4LSX2MWA' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 50
print('Your credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
#print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentials:
CLIENT_ID: MGHGZ0OLVSC4UT1XM4Z5IEH0UKRQ5YOT5XZ42SLUWXDDPUEK


## Pick somewhere central downtown in Shanghai

In [262]:
address = '1331 Fuxing Zhong Lu, Shanghai'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

#Completely innaccurate, sadly
print ('FourSquare is _wrong_ *CORRECT RESULTS ARE* : 31.211398,121.453289')
#Actual Longtitude / Lat is
# 31.211398,121.453289 
latitude = 31.211398
longtitude = 121.453289 



31.2238233 121.4858025
FourSquare is _wrong_ *CORRECT RESULTS ARE* : 31.211398,121.453289


## Search for restaurant venues in this area

##Search for restaurants in the area

> `https://api.foursquare.com/v2/venues/`**search**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&ll=`**LATITUDE**`,`**LONGITUDE**`&v=`**VERSION**`&query=`**QUERY**`&radius=`**RADIUS**`&limit=`**LIMIT**

In [263]:
search_query = 'Restaurant'
#5km's as FourSquare is rather empty for Shanghai
radius = 5000
print(search_query + ' .... OK!')

Restaurant .... OK!


In [265]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=MGHGZ0OLVSC4UT1XM4Z5IEH0UKRQ5YOT5XZ42SLUWXDDPUEK&client_secret=COFCZNWCRD1HKCRIJ5B3M1XP1TLQ04HEVPJASFWO1PGILHSH&ll=31.211398,121.4858025&oauth_token=0JXEDHYX0LFM1M3P1XLVEV0TNGWOGPVCE4PKVR3E4LSX2MWA&v=20180604&query=Restaurant&radius=5000&limit=50'

In [266]:
results = requests.get(url).json()
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)

display_html(dataframe.head())
print ("Row/Col Results: ",dataframe.shape)

,id,name,categories,referralId,hasPerk,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.city,location.state,location.country,location.formattedAddress,location.address,location.crossStreet,location.postalCode,location.neighborhood,venuePage.id
0,57219542498eefea55c61543,陈记海鲜酒家 Chen Ji Seafood Restaurant,"[{'id': '4bf58dd8d48988d1ce941735', 'name': 'S...",v-1627403525,False,31.201891,121.482660,"[{'label': 'display', 'lat': 31.201891, 'lng':...",1099,CN,上海市,上海市,中国,"[上海市, 上海市]",NaN,NaN,NaN,NaN,NaN
1,4bce8945fb84c9b6f20f243e,Cantonese Cuisine Restaurant (家有靓场),"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",v-1627403525,False,31.209735,121.477438,"[{'label': 'display', 'lat': 31.209735, 'lng':...",817,CN,上海市,上海市,中国,"[丽园路501, 上海市, 上海市]",丽园路501,NaN,NaN,NaN,NaN
2,51ebd57b498e5a632e8bb90a,M'Z Restaurant & Café,"[{'id': '4bf58dd8d48988d116941735', 'name': 'B...",v-1627403525,False,31.203032,121.477878,"[{'label': 'display', 'lat': 31.20303195908394...",1198,CN,上海市,上海市,中国,"[局门路550号6102室, 上海市, 上海市]",局门路550号6102室,NaN,NaN,NaN,NaN
3,4f02f82830f899233531a663,DaMaTou Tea Restaurant (大马头茶餐厅),"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",v-1627403525,False,31.220979,121.478385,"[{'label': 'display', 'lat': 31.22097896730746...",1279,CN,Huangpu,上海市,中国,"[西藏南路577号 (近复兴东路), Huangpu, 上海市, 200021]",西藏南路577号,近复兴东路,200021,NaN,NaN
4,4b0588bef964a52050d722e3,Cha's Restaurant (查餐厅),"[{'id': '52af3add3cf9994f4e043bf5', 'name': 'H...",v-1627403525,False,31.220636,121.461609,"[{'label': 'display', 'lat': 31.22063626268351...",2522,CN,Huangpu,上海市,中国,"[30 Sinan Rd, 1F | 思南路30-4号 (Huahai Rd | 淮海中路)...","30 Sinan Rd, 1F | 思南路30-4号",Huahai Rd | 淮海中路,200020,NaN,NaN


Row/Col Results:  (50, 19)


## Tidy up Data

In [267]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
venues = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
venues['categories'] = venues.apply(get_category_type, axis=1)

# clean column names by keeping only last term
venues.columns = [column.split('.')[-1] for column in venues.columns]

print ("Show the Panda Data Store for 'venues' DataStore with Venue Names and location data")
display_html(venues.head(10))
print ("Row/Col Results: ",venues.shape)

Show the Panda Data Store for 'venues' DataStore with Venue Names and location data


,name,categories,lat,lng,labeledLatLngs,distance,cc,city,state,country,formattedAddress,address,crossStreet,postalCode,neighborhood,id
0,陈记海鲜酒家 Chen Ji Seafood Restaurant,Seafood Restaurant,31.201891,121.482660,"[{'label': 'display', 'lat': 31.201891, 'lng':...",1099,CN,上海市,上海市,中国,"[上海市, 上海市]",NaN,NaN,NaN,NaN,57219542498eefea55c61543
1,Cantonese Cuisine Restaurant (家有靓场),Chinese Restaurant,31.209735,121.477438,"[{'label': 'display', 'lat': 31.209735, 'lng':...",817,CN,上海市,上海市,中国,"[丽园路501, 上海市, 上海市]",丽园路501,NaN,NaN,NaN,4bce8945fb84c9b6f20f243e
2,M'Z Restaurant & Café,Bar,31.203032,121.477878,"[{'label': 'display', 'lat': 31.20303195908394...",1198,CN,上海市,上海市,中国,"[局门路550号6102室, 上海市, 上海市]",局门路550号6102室,NaN,NaN,NaN,51ebd57b498e5a632e8bb90a
3,DaMaTou Tea Restaurant (大马头茶餐厅),Chinese Restaurant,31.220979,121.478385,"[{'label': 'display', 'lat': 31.22097896730746...",1279,CN,Huangpu,上海市,中国,"[西藏南路577号 (近复兴东路), Huangpu, 上海市, 200021]",西藏南路577号,近复兴东路,200021,NaN,4f02f82830f899233531a663
4,Cha's Restaurant (查餐厅),Hong Kong Restaurant,31.220636,121.461609,"[{'label': 'display', 'lat': 31.22063626268351...",2522,CN,Huangpu,上海市,中国,"[30 Sinan Rd, 1F | 思南路30-4号 (Huahai Rd | 淮海中路)...","30 Sinan Rd, 1F | 思南路30-4号",Huahai Rd | 淮海中路,200020,NaN,4b0588bef964a52050d722e3
5,Mari e Monti Italian Restaurant,Italian Restaurant,31.209967,121.472489,"[{'label': 'display', 'lat': 31.209967, 'lng':...",1277,CN,上海市,上海市,中国,"[Mengzi Road (Xujiahui Road), 上海市, 上海市]",Mengzi Road,Xujiahui Road,NaN,NaN,4e76de76c65bb91db4fd8b1c
6,新旺茶餐厅 Xinwang Restaurant,Chinese Restaurant,31.207974,121.465385,"[{'label': 'display', 'lat': 31.20797425388117...",1980,CN,上海市,上海市,中国,"[卢湾区徐家汇路550弄1号宝鼎大厦101室 (打浦店), 上海市, 上海市]",卢湾区徐家汇路550弄1号宝鼎大厦101室,打浦店,NaN,NaN,4b9c843ff964a520d06d36e3
7,Chao Man Restaurant (潮民冰室) (潮民冰室),Hong Kong Restaurant,31.226804,121.473690,"[{'label': 'display', 'lat': 31.226804, 'lng':...",2066,CN,Huangpu,上海市,中国,"[Huangpu, 上海市]",NaN,NaN,NaN,NaN,58be8f2a5490d37e4252b13f
8,Qian Restaurant (黔香阁),Guizhou Restaurant,31.226825,121.473713,"[{'label': 'display', 'lat': 31.22682469090645...",2067,CN,Huangpu,上海市,中国,[93-111 Huaihai M Rd | 淮海中路93-111号 (Times Squa...,93-111 Huaihai M Rd | 淮海中路93-111号,Times Square | 大上海时代广场,200021,Huáihǎi Zhōnglù,5b0a25db418686002c623488
9,Super Star Seafood Restaurant (鸿星荟海鲜酒家),Seafood Restaurant,31.225057,121.469323,"[{'label': 'display', 'lat': 31.22505721460541...",2184,CN,Huangpu,上海市,中国,[300 Huaihai M Rd | 淮海中路300号 (4/F-05 K11 Art M...,300 Huaihai M Rd | 淮海中路300号,4/F-05 K11 Art Mall | 上海K11购物艺术中心,200021,Huáihǎi Zhōnglù,5129a540e4b040b1d138f2e6


Row/Col Results:  (50, 16)


## Lets add the ratings to the Venues

In [272]:

#create a new column called rating, and set the default value
venues['rating'] = 0

#loop through our venue's array, and search each one for ratings
for index in venues.index:
    venue_id = venues['id'][index]

    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&oauth_token={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN, VERSION)

    rating = requests.get(url).json()
    try:
        print("Name: ", result['response']['venue']['name'])
        print("Rating: ",result['response']['venue']['rating'])
        #update the rating for this result
        venues['rating'][index]=result['response']['venue']['rating']

    except:
        print('Venue index',index,'is unrated')
        
    #print (rating)
    #print (index)




Venue index 0 is unrated
Venue index 1 is unrated
Venue index 2 is unrated
Venue index 3 is unrated
Venue index 4 is unrated
Venue index 5 is unrated
Venue index 6 is unrated
Venue index 7 is unrated
Venue index 8 is unrated
Venue index 9 is unrated
Venue index 10 is unrated
Venue index 11 is unrated
Venue index 12 is unrated
Venue index 13 is unrated
Venue index 14 is unrated
Venue index 15 is unrated
Venue index 16 is unrated
Venue index 17 is unrated
Venue index 18 is unrated
Venue index 19 is unrated
Venue index 20 is unrated
Venue index 21 is unrated
Venue index 22 is unrated
Venue index 23 is unrated
Venue index 24 is unrated
Venue index 25 is unrated
Venue index 26 is unrated
Venue index 27 is unrated
Venue index 28 is unrated
Venue index 29 is unrated
Venue index 30 is unrated
Venue index 31 is unrated
Venue index 32 is unrated
Venue index 33 is unrated
Venue index 34 is unrated
Venue index 35 is unrated
Venue index 36 is unrated
Venue index 37 is unrated
Venue index 38 is unra

## Show the names,id's of the venues, in descending rated order

In [273]:
newdf = venues[['name','id','rating']]
display_html(newdf.sort_values(by='rating', ascending=False))
highest_rating_id = newdf['id'][0]


,name,id,rating
0,陈记海鲜酒家 Chen Ji Seafood Restaurant,57219542498eefea55c61543,0
37,100 Century Avenue Restaurant | 世纪100,4beec471e8c3c9288c169992,0
27,Memory Restaurant (年代秀),5254dfe3498e1ef2516e55c6,0
28,aliado coffee restaurant (阿利与艾德咖啡西餐厅),4f962c54e4b0d010ab49928e,0
29,ROOF 325 Rooftop Restaurant & Bar,4b0588c1f964a52020d822e3,0
30,Maya Mexican Restaurant (玛雅),4b0fd6f5f964a5205a6523e3,0
31,Lisboa Restaurant (葡京茶餐厅),4bee1d65a40fc9284d19830c,0
32,Wow-Say Sichuan Restaurant (瓦晒川菜),55e4451d498e84aea4f59952,0
33,Han Mama Taiwanese Restaurant (韓媽媽),5a7a75a82f97ec4409fc75eb,0
34,Shanghai No.1 Rest. (上海1号私藏菜),4b74dbe3f964a5209df52de3,0


## Draw Restaurants on Map

In [276]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=14) # generate map centred around the  复兴中路

# add the FourSquare Data as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

## Show the best Venue

##Lets explore around this location


In [281]:
print ("Highest Rated ID: ", highest_rating_id) #Sorted already by highest rating

#Get the array index
index = venues.loc[venues['id'] == highest_rating_id].index[0]
latitude = venues['lat'][index]
longitude = venues ['lng'][index]
name = venues['name'][index]
radius = 2000 #1km
print ("Best Venue Name: ", venues['name'][index])

#Make our call
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}&query="Hotel"'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

Highest Rated ID:  57219542498eefea55c61543
Best Venue Name:  陈记海鲜酒家 Chen Ji Seafood Restaurant


'https://api.foursquare.com/v2/venues/explore?client_id=MGHGZ0OLVSC4UT1XM4Z5IEH0UKRQ5YOT5XZ42SLUWXDDPUEK&client_secret=COFCZNWCRD1HKCRIJ5B3M1XP1TLQ04HEVPJASFWO1PGILHSH&ll=31.201891,121.48266&v=20180604&radius=2000&limit=50&query="Hotel"'

## Now find the best Hotel in the area (using explore)

In [283]:
import requests

results = requests.get(url).json()
'There are {} Hotel results around {}.'.format(len(results['response']['groups'][0]['items']),name)




'There are 19 Hotel results around 陈记海鲜酒家 Chen Ji Seafood Restaurant.'

In [284]:
items = results['response']['groups'][0]['items']


## Convert those to Dataframe and display the first 5

In [287]:
dataframe = json_normalize(items) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# filter the category for each row
dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean columns
dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]


display_html (dataframe_filtered.head(5))




,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id
0,Pullman Shanghai Skyway Hotel,Hotel,"打浦路15号, 浦东新区",Xujiahui Rd.,31.204089,121.471604,"[{'label': 'display', 'lat': 31.204089, 'lng':...",1080,200023,CN,上海市,上海市,中国,"[打浦路15号, 浦东新区 (Xujiahui Rd.), 上海市, 上海市, 200023...",NaN,4bc7f52092b376b0eec1503a
1,Intercontinental Shanghai Expo Hotel (上海世博洲际酒店),Hotel,1188 Xueye Rd,Shibo Avenue,31.200941,121.500556,"[{'label': 'display', 'lat': 31.20094111230248...",1707,NaN,CN,Huangpu,上海市,中国,"[1188 Xueye Rd (Shibo Avenue), Huangpu, 上海市, 中国]",NaN,50360802e4b098c76ebd50a7
2,Shanghai Marriott Riverside Hotel (上海绿地万豪酒店),Hotel,江滨路99号,NaN,31.193658,121.468057,"[{'label': 'display', 'lat': 31.19365803154623...",1665,200023,CN,上海市,上海市,中国,"[江滨路99号, 上海市, 上海市, 200023, 中国]",NaN,4de9c4e088774880e30b1eea
3,Ji Hotel (全季酒店),Hotel,"肇嘉浜路221号, 徐汇区",NaN,31.202784,121.463229,"[{'label': 'display', 'lat': 31.202784, 'lng':...",1852,200032,CN,上海市,上海市,中国,"[肇嘉浜路221号, 徐汇区, Lúwān, 上海市, 上海市, 200032, 中国]",Lúwān,5249242d11d29c723b7fc0bb
4,上海黄金岛大酒店,Hotel,制造局路,NaN,31.201406,121.481498,"[{'label': 'display', 'lat': 31.20140588577529...",123,NaN,CN,Huangpu,上海市,中国,"[制造局路, Huangpu, 上海市, 中国]",NaN,4e5644cbb0fbb294aaa5a0a1


## Draw Hotel Map

In [288]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco


# add Ecco as a red circle mark
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup=name,
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)


# add popular spots to the map as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(venues_map)

# display map
venues_map

In [289]:
dataframe_filtered.head()


,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id
0,Pullman Shanghai Skyway Hotel,Hotel,"打浦路15号, 浦东新区",Xujiahui Rd.,31.204089,121.471604,"[{'label': 'display', 'lat': 31.204089, 'lng':...",1080,200023,CN,上海市,上海市,中国,"[打浦路15号, 浦东新区 (Xujiahui Rd.), 上海市, 上海市, 200023...",NaN,4bc7f52092b376b0eec1503a
1,Intercontinental Shanghai Expo Hotel (上海世博洲际酒店),Hotel,1188 Xueye Rd,Shibo Avenue,31.200941,121.500556,"[{'label': 'display', 'lat': 31.20094111230248...",1707,NaN,CN,Huangpu,上海市,中国,"[1188 Xueye Rd (Shibo Avenue), Huangpu, 上海市, 中国]",NaN,50360802e4b098c76ebd50a7
2,Shanghai Marriott Riverside Hotel (上海绿地万豪酒店),Hotel,江滨路99号,NaN,31.193658,121.468057,"[{'label': 'display', 'lat': 31.19365803154623...",1665,200023,CN,上海市,上海市,中国,"[江滨路99号, 上海市, 上海市, 200023, 中国]",NaN,4de9c4e088774880e30b1eea
3,Ji Hotel (全季酒店),Hotel,"肇嘉浜路221号, 徐汇区",NaN,31.202784,121.463229,"[{'label': 'display', 'lat': 31.202784, 'lng':...",1852,200032,CN,上海市,上海市,中国,"[肇嘉浜路221号, 徐汇区, Lúwān, 上海市, 上海市, 200032, 中国]",Lúwān,5249242d11d29c723b7fc0bb
4,上海黄金岛大酒店,Hotel,制造局路,NaN,31.201406,121.481498,"[{'label': 'display', 'lat': 31.20140588577529...",123,NaN,CN,Huangpu,上海市,中国,"[制造局路, Huangpu, 上海市, 中国]",NaN,4e5644cbb0fbb294aaa5a0a1


In [293]:

k=6
hotels = dataframe_filtered[dataframe_filtered['categories'].str.contains('Hotel',regex=False)]
#name	categories	address	lat	lng	labeledLatLngs	distance	postalCode	cc	city	state	country	formattedAddress	crossStreet	neighborhood	id
hotel_clustering = hotels.drop(['name','categories','address','labeledLatLngs','distance','postalCode','cc','city','state','country','formattedAddress','crossStreet','neighborhood','id'],1)
kmeans = KMeans(n_clusters = k,random_state=0).fit(hotel_clustering)

print ("KMEANS array", kmeans.labels_)

hotels.insert(0, 'KMeansCluster', kmeans.labels_)

hotels

KMEANS array [5 3 0 0 5 5 3 2 0 0 4 1 2 2 1 4 0]


,KMeansCluster,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id
0,5,Pullman Shanghai Skyway Hotel,Hotel,"打浦路15号, 浦东新区",Xujiahui Rd.,31.204089,121.471604,"[{'label': 'display', 'lat': 31.204089, 'lng':...",1080,200023,CN,上海市,上海市,中国,"[打浦路15号, 浦东新区 (Xujiahui Rd.), 上海市, 上海市, 200023...",NaN,4bc7f52092b376b0eec1503a
1,3,Intercontinental Shanghai Expo Hotel (上海世博洲际酒店),Hotel,1188 Xueye Rd,Shibo Avenue,31.200941,121.500556,"[{'label': 'display', 'lat': 31.20094111230248...",1707,NaN,CN,Huangpu,上海市,中国,"[1188 Xueye Rd (Shibo Avenue), Huangpu, 上海市, 中国]",NaN,50360802e4b098c76ebd50a7
2,0,Shanghai Marriott Riverside Hotel (上海绿地万豪酒店),Hotel,江滨路99号,NaN,31.193658,121.468057,"[{'label': 'display', 'lat': 31.19365803154623...",1665,200023,CN,上海市,上海市,中国,"[江滨路99号, 上海市, 上海市, 200023, 中国]",NaN,4de9c4e088774880e30b1eea
3,0,Ji Hotel (全季酒店),Hotel,"肇嘉浜路221号, 徐汇区",NaN,31.202784,121.463229,"[{'label': 'display', 'lat': 31.202784, 'lng':...",1852,200032,CN,上海市,上海市,中国,"[肇嘉浜路221号, 徐汇区, Lúwān, 上海市, 上海市, 200032, 中国]",Lúwān,5249242d11d29c723b7fc0bb
4,5,上海黄金岛大酒店,Hotel,制造局路,NaN,31.201406,121.481498,"[{'label': 'display', 'lat': 31.20140588577529...",123,NaN,CN,Huangpu,上海市,中国,"[制造局路, Huangpu, 上海市, 中国]",NaN,4e5644cbb0fbb294aaa5a0a1
5,5,Magnotel,Hotel,斜土路 | Xietu Rd.,Mangzi Lu,31.204249,121.474147,"[{'label': 'display', 'lat': 31.20424914351182...",851,NaN,CN,Huangpu,上海市,中国,"[斜土路 | Xietu Rd. (Mangzi Lu), Huangpu, 上海市, 中国]",NaN,4cac7781a6e08cfa165aab94
6,3,URSIDE Hotel & Café,Hotel,"Unit 3A, Block 4, 60 Huayuangang Road | 花园港路60...",Ban songyuan Road | 半淞虹园路,31.204022,121.492352,"[{'label': 'display', 'lat': 31.20402151063051...",952,NaN,CN,Huangpu,上海市,中国,"[Unit 3A, Block 4, 60 Huayuangang Road | 花园港路6...",NaN,5bd43706e55d8b002ce1eb2a
7,2,全季酒店 Ji Hotel,Hotel,"重庆南路283号, 黄浦区",徐家汇路,31.209369,121.473307,"[{'label': 'display', 'lat': 31.209369, 'lng':...",1219,200025,CN,上海市,上海市,中国,"[重庆南路283号, 黄浦区 (徐家汇路), 上海市, 上海市, 200025, 中国]",NaN,4d0ce162bdbfa35d7e436672
8,0,Atour S Hotel (Atour S Hotel 亚朵酒店),Hotel,斜土路688号,上海市卢湾区斜土路688号,31.204238,121.468373,"[{'label': 'display', 'lat': 31.20423812686419...",1385,NaN,CN,Huangpu,上海市,中国,"[斜土路688号 (上海市卢湾区斜土路688号), Huangpu, 上海市, 中国]",NaN,4dbd303443a1d8504b9f67a0
9,0,Ramada Peace Hotel Shanghai,Hotel,679号 Xujiahui Rd,NaN,31.204445,121.468261,"[{'label': 'display', 'lat': 31.20444534386017...",1400,NaN,CN,Huangpu,上海市,中国,"[679号 Xujiahui Rd, Huangpu, 上海市, 中国]",NaN,4bf08355b315c9b6e61d93ff


## Draw our KMeans Cluster map

In [294]:

# create map
map_clusters = folium.Map(location=[latitude,longitude],zoom_start=14)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lng, name, cluster, address in zip(hotels['lat'], hotels['lng'], hotels['name'], hotels['KMeansCluster'], hotels['address']):
    label = ' Cluster ' + str(cluster)+ ' {}, {}'.format(name, address)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        color='red',
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters
